# Ejemplo de consulta de datos en PI

In [1]:
from datetime import datetime, timedelta

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [3]:
import re

In [4]:
import clr
clr.AddReference('System.Collections')
from System.Collections.Generic import List

In [5]:
# Sólo para cuando se ejecuta desde notebooks:
import warnings
warnings.filterwarnings("ignore", category=Warning)
# Seteos de Pandas para ver mejor la info. https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
# In max_columns and max_rows ‘None’ value means unlimited.
pd.set_option('display.max_rows', 100)
# Conviene setear min_rows también porque cuando se superan las max_rows, sólo muestra la cantidad indicada en min_rows.
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:.1f}'.format

In [6]:
path_data = '.\data'

# (1) Real Time PI utilizando PIConnect
Este código consulda datos según la estructura de AF. No consulta únicamente datos de tiempo real.<br>
Para poder consultar, se necesita conocer la estructura de arbol definida en AF.<br>

__[Parte 1]__<br>
Se puede definir una lista de "padres": parents_lst (deben ser todos del mismo template, o sea, tener los mismos atributos), de los cuales se quiera una lista de "atributos": parents_att_lst. <br>
También se puede definir una lista de "elementos" o descendientes: elements_lst. Si se definió una lista de padres, la lista de descendientes se espera que sean descendientes de estos padres. Si no se definió una lista de padres, se puede definir una lista de elementos sueltos (deben ser todos del mismo template, es decir, compartir atributos). Y una lista de atributos, si no queremos todos.
Parte 1: Este código obtiene los datos a consultar.<br>
__[Parte 2]__<br>
Indicar la fecha desde. Esta parte del código arma un período de tiempo entre la fecha desde y ahora (* en PI)<br>
__[Parte 3]__<br>
Consulta los datos con una determinada frecuencia. En el ejemplo, consulta los datos cada 2 min. Hay que indicar este valor (2 min) en 2 lugares del código:<br>
`intervals = (7 * 24 * 60)/2`<br>
`time_span = PI.AFSDK.AF.Time.AFTimeSpan.Parse('2m')`<br>
Los datos quedan en un dataframe `df_element_data_raw`

### Librerías necesarias

In [ ]:
# pip install PIconnect
# NOTA: chequear si requiere instalar pythonnet, yo instalé PIConnect en un ambiente que ya tenía pythonnet

In [7]:
import PIconnect as PI
#from PIconnect.PIConsts import RetrievalMode # retrieval_mode=RetrievalMode.AT_OR_BEFORE

OSIsoft(r) AF SDK Version: 2.10.9.593


In [8]:
pi_server_name = 'PARCCVWPIEDA01' # sys.argv[1] #
pi_database_name = 'DOF' # sys.argv[2] # 

PIdatabase = PI.PIAFDatabase(server=pi_server_name, database=pi_database_name)
for root in PIdatabase.children.values():
    att = root.attributes.get('Fechas - Fecha Parte Operativo Zafiro').attribute
    AFDatabase = att.Database

In [9]:
def treat_element(element, parent_data_att_lst, elements_att_lst, elements_data_att_lst):
    """ Builds the element record, using att from parent and adding att from current element
        Adds the element object attribute list to the general attribute list to query real time data
        element_data_att_obj_lst must be globally declared and type: PI.AFSDK.AF.Asset.AFAttributeList()
    """
    # Use the parent data on the element record:
    _element_data_lst = parent_data_att_lst.copy()
    # Get element data -no real time
    for att in elements_att_lst:
        # User should have asked to include the PI path of the element, by including 'Path' or 'path' in the list of static data att:
        if 'path' == att.lower():
            _element_data_lst.append(element.GetPath())
        # User should have asked to include the PI internal ID of the element, by including 'element_id' in the list of static data att:
        elif 'element_id' == att.lower():
            _element_data_lst.append(element.ID)
        elif 'name' == att.lower():
            _element_data_lst.append(element.Name)
        else:
            att_value = element.Attributes.get_Item(att).GetValue().Value
            att_is_good = element.Attributes.get_Item(att).GetValue().IsGood
            _element_data_lst.extend([att_value if att_is_good else np.nan])
    # REAL TIME DATA: build the list of att for the recorded data at times query
    for data_att in elements_data_att_lst:
        element_data_att_obj_lst.Add(element.Attributes.get_Item(data_att))
    return _element_data_lst

In [10]:
#[[[[[[[[[[[[[[[[[[ PARTE 1 ]]]]]]]]]]]]]]]]]]
#------------------------------------------------------------------------------------------------------------
# List of parents - keep null if you dont need to query all elements dependent of a father element
parents_lst = ['\\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores']
#parents_lst = []
# List of att from each parent. Not mandatory
parents_att_lst = ['SAP Ubicación Técnica']
parents_att_lst = []
#------------------------------------------------------------------------------------------------------------
# List of elements. Keep null if you want to query all descendants of each parent element from parent_lst
elements_lst = ['\\PAMPA ENERGIA S.A.\Sierra Chata\Instalaciones\EMC Zona Norte\Compresores\K-101 CZN SCH']
elements_lst = ['K-02']
# List statict data from element
# IMPORTANT: User should have asked to include the PI path of the element, by including 'Path' or 'path' in the list of static data att
# Same way, user should have asked to include the PI internal ID of the element, by including 'element_id' in the list of static data att:
elements_att_lst = ['Path', 'element_id', 'Name', 'ID Equipo', 'Equipo']
# List real time data from element
elements_data_att_lst = ['Carga Motriz Compresor', 'Presion Descarga Cil. 1y3 Compresor', 'Presion Descarga Cil. 2y4 Compresor'
                        , 'Presion Succion Compresor', 'RPMs Instantanea']
#elements_data_att_lst = ['RPMs Instantanea']

#------------------------------------------------------------------------------------
# In order to define the list of fields for the final ELEMENT dataframe (static data):
fields_lst = ['element_path_pi', 'element_id_pi', 'element', 'facility_name', 'facility_id_za']

#------------------------------------------------------------------------------------
# In order to define the list of fields for the final DATA dataframe (real time data):
feat_lst = ['carga_motriz', 'press_1', 'press_2', 'press_suc', 'rpm']

# List of data att. As an AFAttributeList() object so you can use its method Data.RecordedValuesAtTimes()
element_data_att_obj_lst = PI.AFSDK.AF.Asset.AFAttributeList()

if parents_lst:
    # get the parent attibutes in a python list, so you can add parent data to the elements list
    for parent in parents_lst:
        parent_obj = AFDatabase.Elements.GetItem(parent, True, True, True)
        parent_data_att_lst = []
        for parent_att in parents_att_lst:
            att_value = parent_obj.Attributes.get_Item(parent_att).GetValue().Value
            att_is_good = parent_obj.Attributes.get_Item(parent_att).GetValue().IsGood
            ##### PENDING: functionality to change the type of the att (sometimes IDs comes as System.Double and should be integer)
            ##### You can add a list of types and cast to that type.
            parent_data_att_lst.extend([att_value if att_is_good else np.nan])
        if elements_lst:
            elements_data_lst = []
            # just bring data from the list of elements defined by user
            for element in elements_lst:
                element_obj = AFDatabase.Elements.GetItem(parent + '\\' + element, True, True, True)
                element_data_lst = treat_element(element_obj, parent_data_att_lst, elements_att_lst, elements_data_att_lst)
                # list to build the dataframe of static data of elements
                elements_data_lst.append(element_data_lst)
        else:
            # Bring data from all the desendants.
            i=0
            elements_data_lst = []
            for element in parent_obj.Elements:
                i+=1
                if True:
                    element_data_lst = treat_element(element, parent_data_att_lst, elements_att_lst, elements_data_att_lst)
                    # list to build the dataframe of static data of elements
                    elements_data_lst.append(element_data_lst)
else:
    if elements_lst:
        # No parent defined --> Use only parent name
        # just bring data from the list of elements defined by user
        elements_data_lst = []
        # just bring data from the list of elements defined by user
        for element in elements_lst:
            element_obj = AFDatabase.Elements.GetItem(element, True, True, True)
            parent_obj = element_obj.Parent
            # Name is an attibute every asset has:
            parent_att_lst = ['Name']
            parent_data_att_lst = [parent_obj.Name]
            element_data_lst = []
            element_data_lst = treat_element(element_obj, parent_data_att_lst, elements_att_lst, elements_data_att_lst)
            # list to build the dataframe of elements (only no real time data)
            elements_data_lst.append(element_data_lst)
    else:
        # Should define any element!
        print('Ups! Elements Not Defined. Should define any element!')
df_element = pd.DataFrame(data=elements_data_lst, columns=fields_lst)
df_element

,element_path_pi,element_id_pi,element,facility_name,facility_id_za
0,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,9417996f-f828-11ea-813a-0050569d6e61,K-02,26,K-02


In [11]:
#[[[[[[[[[[[[[[[[[[ PARTE 2 ]]]]]]]]]]]]]]]]]]
pagingconfig = PI.AFSDK.AF.PI.PIPagingConfiguration(PI.AFSDK.AF.PI.PIPageType.TagCount, 100)
#------------------------------------------------------------------------------------------------------------
# Build Time range and span.
exact_hr_timerange_end = datetime(PI.AFSDK.AF.Time.AFTime.Now.LocalTime.Year
                                , PI.AFSDK.AF.Time.AFTime.Now.LocalTime.Month
                                , PI.AFSDK.AF.Time.AFTime.Now.LocalTime.Day
                                , PI.AFSDK.AF.Time.AFTime.Now.LocalTime.Hour, 0, 0)
timerange_end = PI.AFSDK.AF.Time.AFTime(exact_hr_timerange_end.strftime("%d/%m/%Y, %H:%M:%S"))

#[[[[[[[[[[[[[[[[[[ INDICAR FECHA DESDE ]]]]]]]]]]]]]]]]]]
timerange_start = PI.AFSDK.AF.Time.AFTime('01/11/2022 00:00:00')

timerange = PI.AFSDK.AF.Time.AFTimeRange(timerange_start, timerange_end)
print('Start', timerange_start, 'End:', timerange_end, 'Time Range', timerange)

span = PI.AFSDK.AF.Time.AFTimeSpan.Parse('2m')
print("TimeSpan ", span)
print('Number of Intervals:', span.GetEvenTimeIntervalDefinitions(timerange)[0].NumberOfIntervals)
#116532

Start 1/11/2022 00:00:00 End: 7/11/2022 16:00:00 Time Range 1/11/2022 00:00:00 - 7/11/2022 16:00:00
TimeSpan  2m
Number of Intervals: 4800


In [12]:
def build_df_at_times(times, element_data_att_obj_lst, pagingconfig):
    # Get recorded values AT TIMES (At Or Before):
    recorded_values_lst = element_data_att_obj_lst.Data.RecordedValuesAtTimes(times
                                        , PI.AFSDK.AF.Data.AFRetrievalMode.AtOrBefore, pagingconfig)
    # Build dataframe: WARNING you have tu use as timestamp the times calculated earlier.
    rows = []
    idx=0
    for values in recorded_values_lst:
        idx+=1
        if True: #idx <10:
            i=0
            for value in values:
                rows.append([str(values.Attribute.Element.ID)
                            , values.Attribute.Element.GetPath()
                            , datetime.strptime(times[i].ToString(), '%d/%m/%Y %H:%M:%S')
                            , value.Attribute.Name, value.Value])
                i+=1
    return pd.DataFrame(data=rows, columns=['element_id_pi', 'element_path_pi', 'timestamp', 'attribute', 'value'])


In [13]:
#[[[[[[[[[[[[[[[[[[ PARTE 3 ]]]]]]]]]]]]]]]]]]
start = datetime.now()

# Example: Quantity of intervals of 2 min in a week:
intervals = (7 * 24 * 60)/2
time_span = PI.AFSDK.AF.Time.AFTimeSpan.Parse('2m')
time_init = timerange_start
# list of dataframes to concat at the end of loops
df_lst = []
for interval in range(0, round(time_span.GetEvenTimeIntervalDefinitions(timerange)[0].NumberOfIntervals/intervals)-1):
    # Build a LIST (C#) of AFTime objects for each hour:
    times = List[PI.AFSDK.AF.Time.AFTime]()
    time_end = time_span.Multiply(time_init, intervals)
    print(time_init.ToString(), time_end.ToString())
    timerange_loop = PI.AFSDK.AF.Time.AFTimeRange(time_init, time_end)
    times.Add(time_init)
    time_aux = time_init
    for t in range(1, time_span.GetEvenTimeIntervalDefinitions(timerange_loop)[0].NumberOfIntervals):
        time_aux = PI.AFSDK.AF.Time.AFTime.op_Addition(time_aux, time_span)
        times.Add(time_aux)
    # Get recorded values AT TIMES (At Or Before) and build dataframe:
    df_lst.append(build_df_at_times(times, element_data_att_obj_lst, pagingconfig))
    time_init = time_span.Multiply(time_end, 1)
# last loop 'till *
times = List[PI.AFSDK.AF.Time.AFTime]()
time_end = timerange_end
print(time_init.ToString(), time_end.ToString())
timerange_loop = PI.AFSDK.AF.Time.AFTimeRange(time_init, time_end)
times.Add(time_init)
time_aux = time_init
for t in range(1, time_span.GetEvenTimeIntervalDefinitions(timerange_loop)[0].NumberOfIntervals):
    time_aux = PI.AFSDK.AF.Time.AFTime.op_Addition(time_aux, time_span)
    times.Add(time_aux)
df_lst.append(build_df_at_times(times, element_data_att_obj_lst, pagingconfig))
df_element_data_raw = pd.concat(df_lst)

1/11/2022 00:00:00 7/11/2022 16:00:00


In [14]:
#######################################################################
# Check records with data errors:
df_values_err = df_element_data_raw[~df_element_data_raw.value.astype(str).str.match('[0-9]+\.[0-9]+')]
print('Total Registros:', df_element_data_raw.shape[0])
print('Registros con error:', df_values_err.shape[0], '('+str(round(df_values_err.shape[0]/df_element_data_raw.shape[0]*100, 1))+'%)')
print('Tipos de errores (columna value):', df_values_err[['value']].drop_duplicates(subset=['value']))

Total Registros: 24000
Registros con error: 820 (3.4%)
Tipos de errores (columna value):             value
261   I/O Timeout
574     Comm Fail
3490          Bad


In [15]:
# Adjustments:
#------------------------------------------------------------------------------------------------------------
# You can receive a value of type PI.PINET (or something like that) --> we are going to mark those records
# with "PI Point not found":
df_element_data_raw.value = df_element_data_raw.value.apply(lambda x: x if isinstance(x, (int, str, float)) 
                                                          else 'PI Point not found' 
                                                          if 'PI Point not found' in str(x) else x)

# Continue working with a copy:
df_element_data = df_element_data_raw.copy()

# We have to transpose the matrix
df_aux_1 = df_element_data[df_element_data.attribute==elements_data_att_lst[0]]
df_aux_1.rename(columns={'value': feat_lst[0]}, inplace=True)
df_aux_1.drop(columns=['attribute'], inplace=True)
for i in range(1, len(elements_data_att_lst)):
    df_aux_2 = df_element_data[df_element_data.attribute==elements_data_att_lst[i]]
    df_aux_2.rename(columns={'value': feat_lst[i]}, inplace=True)
    df_aux_2.drop(columns=['attribute'], inplace=True)
    df_aux_1 = pd.merge(df_aux_1, df_aux_2,  how='left', left_on=['element_id_pi', 'element_path_pi', 'timestamp']
                    , right_on = ['element_id_pi', 'element_path_pi', 'timestamp'])
    df_aux_1.reset_index(drop=True, inplace=True)

# Adjust Bad, Comm Fail, I/O Timeout and PI Point Not Foud to a negative numeric value code.
for column in df_aux_1.columns[3:]:
    if df_aux_1[column].dtype == 'object':
        df_aux_1[column] = df_aux_1[column].astype(str).replace('Bad', '-1')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('Comm Fail', '-2')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('I/O Timeout', '-3')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('PI Point not found', '-4')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('Not Connect', '-5')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('No Data', '-6')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('Pt Created', '-7')
        df_aux_1[column] = df_aux_1[column].astype(str).replace('Configure', '-8')
        df_aux_1[column] = df_aux_1[column].astype('float64' , copy=True)
df_element_data = df_aux_1.copy()
end = datetime.now()
hours = (end-start).seconds//3600
minutes = ((end-start).seconds - (hours * 3600))//60
seconds = (end-start).seconds%60
print('Execution time: {0:.2f} hours'.format(hours), '{0:.2f} minutes'.format(minutes)
      , '{0:.2f} seconds'.format(seconds),'Rows Generated:', df_element_data.shape[0])
print(df_element_data.shape)
print(df_element_data.dtypes)
df_element_data.head()

Execution time: 0.00 hours 0.00 minutes 27.00 seconds Rows Generated: 4800
(4800, 8)
element_id_pi              object
element_path_pi            object
timestamp          datetime64[ns]
carga_motriz              float64
press_1                   float64
press_2                   float64
press_suc                 float64
rpm                       float64
dtype: object


,element_id_pi,element_path_pi,timestamp,carga_motriz,press_1,press_2,press_suc,rpm
0,9417996f-f828-11ea-813a-0050569d6e61,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,2022-11-01 00:00:00,91.6,84.6,84.8,33.6,1050.4
1,9417996f-f828-11ea-813a-0050569d6e61,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,2022-11-01 00:02:00,92.9,84.6,84.8,33.6,1050.7
2,9417996f-f828-11ea-813a-0050569d6e61,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,2022-11-01 00:04:00,91.1,84.6,84.8,33.6,1052.0
3,9417996f-f828-11ea-813a-0050569d6e61,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,2022-11-01 00:06:00,91.7,84.6,84.8,33.6,1051.4
4,9417996f-f828-11ea-813a-0050569d6e61,\\PARCCVWPIEDA01\DOF\PAMPA ENERGIA S.A.\El Mangrullo\Instalaciones\PTG EMAN\Compresores\K-02,2022-11-01 00:08:00,91.7,84.6,84.8,33.6,1052.1
